In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("trongnguyen04/nckh-2425-crops")
print("Path to dataset files:", path)

# ─── copy raw folder “1” → /content/dataset_original/crops ───────
SRC="/root/.cache/kagglehub/datasets/trongnguyen04/nckh-2425-crops/versions/1"
DEST="/content/dataset_original/crops"
!mkdir -p $(dirname $DEST)
!cp -r "$SRC" "$DEST"

100%|██████████| 107M/107M [00:06<00:00, 18.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/trongnguyen04/nckh-2425-crops/versions/1


In [2]:
!git clone https://github.com/htrnguyen/compare_ocr_benchmark.git

Cloning into 'compare_ocr_benchmark'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 20 (delta 3), reused 19 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 72.75 KiB | 14.55 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [7]:
!pip install python-Levenshtein jiwer

In [1]:
import sys
sys.path.append('/content/compare_ocr_benchmark/common')

from image_preprocessing import preprocess_image
from ocr_base import OCRModelBase
from metrics import compute_metrics
from utils import read_annotations, save_results

import os
import pandas as pd
import time

In [9]:
!pip install vietocr

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.6/299.6 kB 27.7 MB/s eta 0:00:00
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14845 sha256=ccfff84bf5855c192e9063f39c89e3dc7b5a5549eba066353fa2e6e94d9a7725
  Stored in directory: /root/.cache/pip/wheels/04/17/f7/f9585b8ed11a19d05b50f0ab33aa6635a5179241b92982ab27
  Created wheel for prefetch-generator: filename=prefetch_generator-1.0.1-py3-none-any.whl size=3941 sha256=96167ee85445db49b6d78f441ee41abdad5decc8970f24565430e393da2c3508
  Stored in directory: /root/.cache/pip/wheels/8b/24/c0/c552730f6b36b3dfca2

In [2]:
from vietocr.tool.config import Cfg
from vietocr.tool.predictor import Predictor
from PIL import Image

In [3]:
class VietOCRModel(OCRModelBase):
    def __init__(self, config_name='vgg_transformer', device='cuda'):
        super().__init__()
        config = Cfg.load_config_from_name(config_name)
        config['device'] = device
        self.detector = Predictor(config)

    def predict(self, img_pil, img_np=None):
        t1 = time.perf_counter()
        pred = self.detector.predict(img_pil, return_prob=False)
        t2 = time.perf_counter()
        return pred, t2 - t1

In [4]:
CSV_ANN = '/content/dataset_original/crops/crops_gt.csv'
IMG_DIR = '/content/dataset_original/crops'

df = read_annotations(CSV_ANN)

In [5]:
model = VietOCRModel()

results = []
for idx, row in df.iterrows():
    fname = row['filename']
    desc_gt = row['description_gt']
    label = row.get('label', '')
    img_path = os.path.join(IMG_DIR, fname)

    img_pil, img_np = preprocess_image(img_path, target_size=448, do_auto=True)
    pred, infer_time = model.predict(img_pil, img_np)
    metrics = compute_metrics(desc_gt, pred)

    results.append({
        "filename": fname,
        "label": label,
        "ground_truth": desc_gt,
        "predicted_text": pred,
        "cer": metrics["cer"],
        "wer": metrics["wer"],
        "lev": metrics["lev"],
        "acc": metrics["acc"],
        "time": round(infer_time, 3)
    })

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:07<00:00, 79.7MB/s]
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
18533it [00:00, 26519.15it/s]
/usr/local/lib/python3.11/dist-packages/vietocr/tool/translate.py:115: RuntimeWarning: invalid value encountered in divide
  char_probs = np.sum(char_probs, axis=-1)/(char_probs>0).sum(-1)
/usr/local/lib/python3.11/dist-packages/vietocr/tool/translate.py:115: RuntimeWarning: invalid value encountered in divide
  char_probs = np.sum(char_probs, axis=-1)/(char_probs>0).sum(-1)
/usr/local/lib/python3.11/dist-packages/vietocr/tool/translate.py:115: RuntimeWarning: invalid value encountered in divide
  cha

In [6]:
import os
OUT_CSV = '/content/compare_ocr_benchmark/results/vietocr_results.csv'
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
save_results(results, OUT_CSV)

Lưu thành công: /content/compare_ocr_benchmark/results/vietocr_results.csv
